[<<<< Връщане към Задача 1](lession1.ipynb) $~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ [Начална страница](./../welcome.ipynb) $~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ [Продължи към Бонус 2 >>>>](bonus2.ipynb)
# Полезна математика
---

## Бонус 1: Измерване на разстоянието до жилищна сграда от снимка
В предишната задача научи как може да се измерва разстоянието до реален обект от снимка чрез специално за това създаден маркер. Само че не е задължително да се използва специален маркер за това. Освен това в едно реално приложение на метода, примерно за автономна навигация на робот, маркера трябва автоматично да се засече и измери колко е голям. За това съм ти приготвил две бонус задачи. При тези задачи фокуса вече няма да е върху математиката, ами върху това, как по-ефективно да приложим метода за решаването на практически проблеми чрез програмиране.

Съдържание:

1. [Увод](#1-Увод)  
2. [Зареждане на нужните Python библиотеки](#2-Зареждане-на-нужните-Python-библиотеки)  
3. [Зареждане на снимка](#3-Зареждане-на-снимка)  
4. [Начертаване на измервателна линия](#4-Начертаване-на-измервателна-линия)  
5. [Функция за пресмятане на дължината на линията](#5-Функция-за-пресмятане-на-дължината-на-линията)  
6. [Пресмятане на разстоянието Z до сградата](#6-Пресмятане-на-разстоянието-Z-до-сградата)  
7. [Проверка](#7-Проверка)  


### 1 Увод
Метода, който използвахме в миналата задача, може да се ползва и за измерване на разстоянието до случайни обекти, на които знаем размера. На пример при сгради особено подходяща величина е височината на един етаж. За да измерим правилно височината на една сграда или нейн етаж от снимка, трябва вертикалните ръбове на сградата да се виждат на снимката като паралелни линии. 

За целта направих една снимка, на която на заден план се вижда жилищна сграда. Ъгълът между крайните два вертикални ръба на сградата е само около 1°, виж *Фигура 1* по-долу. Тоест ръбовете са почти паралелни и можем да измерим разстоянието от камерата до сградата по снимката.

<img src="bonus1/fig1.png" alt="Фигура 1" width="600"/><br>
*Фигура 1. Отрязък от снимката със сградата, до която ще измерим разстоянието. Вертикалните линии са отбелязани като червени линии.*


#### 2 Зареждане на нужните Python библиотеки
Тъй като това е нова страница, трябва отново да заредим нужните Python библиотеки. Затова кликни върху блока с код по-долу за да се маркира и натисни клавишната комбинация Shift+Enter или бутона Run ![run.png](run.png) от лентата с бутони горе.

In [ ]:
# активиране на интерактивния графичен плот
%matplotlib widget
# зареждане на библиотеката за графичен плот
import matplotlib.pyplot as plt
# зареждане на библиотеката за интерактивни контроли
import ipywidgets as widgets
# зареждане на библиотеката OpenCV за компютърно зрение за обработка на изображение
import cv2
# зареждане на библиотеката за математически изчисления
import math
# зареждане функцията ruler_plt() от модула за измерване на дължини върху плот
from custom_scripts.measure_plt import ruler_plt
# зареждане на функцията read_img() от модула за прочитане на качени снимки
from custom_scripts.read_uploaded_img import read_img
# изкарване на съобщение като се извършат всички команди
print('>>>Готово!<<<')

#### 3 Зареждане на снимка
С блока по-долу можеш да заредиш направената от мен снимка.

In [ ]:
image = cv2.imread('bonus1/pleven.jpg')
# преобразуване в стандартната за matplotlib цветова схема червено/зелено/синьо (RGB)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# изкарване на съобщение като се извършат всички команди
print('>>>Готово!<<<')

Ако искаш можеш да използваш и твоя снимка. За да я качиш, активирай следващия блок. Под него ще се появи бутон "Upload", от който можеш да избереш желаната снимка.

In [ ]:
uploader = widgets.FileUpload(accept='', multiple=False)
display(uploader)

След като качиш твоята снимка, тя трябва да се преобразува в подходящ формат за работа. Затова активирай следващия блок.

In [ ]:
# прочитане на качената снимка
image = read_img(uploader)

#### 4 Начертаване на измервателна линия
Сега трябва да измерим височината на сградата от снимката. Тъй като на моята снимка не се вижда цялата сграда, на нея ще измерим височината само на етажите, които се виждат напълно. Това са последните седем етажа, виж *Фигура 2*. Също има значение, по коя вертикална линия на сградата ще измерим тези седем етажа. На края ще получим разстоянието точно до тази вертикална линия.


<img src="bonus1/fig2.png" alt="Фигура 2" width="600"/><br>
*Фигура 2. Последните седем етажа, които ще използваме за измерването.*


Актвивирай блока по-долу с кода за показване на снимката и начертаване на измервателната линия. Както вече беше споменато, всяко наше действие може да отнеме 1-2 секунди докато се отрази на снимката, за това трябва да работим бавно и търпеливо.

<details>
    <summary>Натисни тук ако искаш да си припомниш стъпките за измерване на дължината L'</summary>
<br>

- стартираме скрипта от долния блок
- под него се показва снимката
- преместваме курсора с мишката върху снимката за да се покаже лента с бутони от лявата страна
- щракаме на бутона с квадрадче ![rect.png](rect.png) "Zoom to rectangle"
- чертаем квадрад около окръжността на снимката за да я увеличим (чрез натискане и задържане на левия бутон на мишката (LMB) и влачене на мишката за да разпънем квадрата)
- ако сме увеличили грешен регион, можем да върнем стъпка назад от бутона със стрелка сочеща на ляво ![back.png](back.png) "Back to previous view"
- поставяш началната и крайната точка на линията чрез натискане на RMB върху снимката на желаното място
- след като вече е начертана линията, при повторно натискане на RMB върху изображението, линията се изтрива и се създава нова начална точка за нова линия
- можеш да поставяш началната и крайната точка на линията на ново толкова пъти, колкото е нужно за да оцелиш точно дължината на елипсата и си доволен с резултата
- като си доволен с резултата, трябва да внимаваш да не кликнеш по невнимание някъде върху снимката с RMB и така да започнеш чертането на линия на ново
- в променливата `coords` се запазват координатите на началната и крайната точка на начертаната линия
</details>

In [ ]:
# създаване на нов плот
fig = plt.figure()
# създаване на подплот
ax = fig.add_subplot(111)
# показване на снимката върху плота
ax.imshow(image)
# активиране на функцията за чертане на линия при кликване върху плота
coords = ruler_plt(fig, ax)

#### 5 Функция за пресмятане на дължината на линията
Тъй като доста често ни се налага да изчислим дължината на линия в 2D пространството, можем да напишем функция, която прави това вместо нас. Все пак сме в среда за програмиране.

Първо ще си припомним още веднъж [питагоровата теорема](https://bg.wikipedia.org/wiki/%D0%9F%D0%B8%D1%82%D0%B0%D0%B3%D0%BE%D1%80%D0%BE%D0%B2%D0%B0_%D1%82%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0) за пресмятане на дължината на линия $c$ от компонентите й $a$ и $b$:

\begin{align}
    c = \sqrt{a^2 + b^2}. \tag{6}
\end{align}

и формулите за пресмятане на компонентите $a$ и $b$ от дадени координати $[(x_1, y_1), (x_2, y_2)]$  на линията:

\begin{align}
    a &= x_2 - x_1, \tag{7}\\
    b &= y_2 - y_1. \tag{8}
\end{align}

Вече на два пъти сме писали код за това пресмятане за дължината на линията $L'$ в предходната [Задача 1](lession1.ipynb#4.4.2-Пресмятане-на-дължината-на-линията), използвайки координатите на линията запаметени в променливата `coords`. Сега обаче използваме $c$ за дължината на линията, и $a$ и $b$ за компонентите й, и за това кода изглежда по следния начин:

```
a = coords[1][0] - coords[0][0]
b = coords[1][1] - coords[0][1]
c = math.sqrt(a**2 + b**2)
```

Сега ни трябва функция, на която да предоставим променливата `coords`, тя автоматично да изпълни трите реда код по-горе и накрая да ни върне като резултат дължината на линията `c`.

За целта трябва първо да дефинираме нова функция използвайки ключовата дума `def`, последвана от име за функцията, примерно `line_length`, от нужната променлива `coords` в скоби и накрая от две точки ":" за завършек:

```
def line_length(coords):
```

С този първи ред само обяваваме и кръщаваме новата функция. Сега следва кода във функцията, който трябва да се изпълнява при всяко извикване на функцията. Кода трябва да има един отстъп (на англ. tab) на вътре и в случая се състои от трите реда код от по-горе.

На края на функцията посочваме с ключът `return`, че искаме да върнем пресметнатата стойност от променливата $c$ като резултат:
```
return c
```

Цялата функция е представена в блока по-долу. Активирай го, за да се активира функцията и за да може да се извиква от други блокове.

In [ ]:
def line_length(coords):
    a = coords[1][0] - coords[0][0]
    b = coords[1][1] - coords[0][1]
    c = math.sqrt(a**2 + b**2)

    return c

Сега можем по следния начин да извикаме новата функция `line_length()` за да пресметнем дължината на линията от променливата `coords` и да запазим резултата примерно в променливата `length`:

In [ ]:
length = line_length(coords)
# изкарай дължината на линията
print('дължина = ', length)

#### 6 Пресмятане на разстоянието Z до сградата
Разстоянието $Z$ от камерата до сградата може да бъде пресметнато чрез познатото ни уравнение $(2)$:

\begin{align}
    Z = f\cdot\frac{L}{L'}. \tag{2}
\end{align}

В това уравнение фокусното разстояние $f$ е вече известно от калибрирането на камерата от точка [4.5](lession1.ipynb#4.5-Пресмятане-на-фокусното-разстояние-$f$) от предходната Задача 1. $L'$ пък е дължината на етажите, измерена в пиксели в предходната подточка 1.4 и запаметена в променливата `length`. Освен това ни трябва и реалната дължина на етажите $L$. Височината на един етаж е различна при различните сгради и освен ако не я измерим собственоръчно, няма как да знаем точно колко е. Затова ще трябва да направим предположение, от което ще зависи точността на измерването. На направената от мен снимка можем да предположим, че височината на един етаж е някъде $2.70$ до $3$ метра, примерно $2.80 \, m$. На нея измерихме височината на седем етажа, съответно и тук трябва да използваме височината на седем етажа за пресмятането на $L$:

\begin{align}
    L = 7 \cdot 2.8 \, m = 19.6 \, m. \tag{9}
\end{align}

Ако си използвал твоя си снимка, то тогава трябва сам да предположиш колко горе долу би бил висок един етаж и да умножиш по броя на измерените етажи.

Сега остава само да въведеш стойностите на $f$ и $L$, да пресметнеш $Z$ и да покажеш резултата чрез функцията print() в следващия блок.

In [ ]:
# Първо трябва да въведеш фокусното разстояние f от предната задача
f = 
# сега въведи предполагаемата височина на етажите
# (ако си използвал моята снимка, то тя е 19.6 метъра за 7 етажа):
L = 
# и извърши пресмятането използвайки вече запаметената променлива length от по-горе:
Z = 
# сега можеш да покажеш резултата чрез функцията print()


<details>
    <summary>Ако не можеш да се сетиш как да напишеш формулата за Z, щракни тук за да видиш отговора.</summary>
<br>

```python
Z = f * L / length
```

</details>
<br>

#### 7 Проверка
За да проверим колко са ни верни изчисленията, можем да измерим въпросното разстояние $Z$ на google maps. Разстоянието до сградата, която аз съм заснел, от мястото на заснемането до измерения ръб на сградата е около $251.3$ метъра, виж по-долу *Фигура 3*. При мен $Z$ се получи, че е $230.5$ метъра. Това е отклонение от около $-21$ метъра, или $(-21 \, m / 251 \, m) \cdot 100 \, \% \approx -8.4 \, \%$. Това хич не е лош резултат, предвид че не знаем точната физическа височина на един етаж и предвид че използваме сравнително прост метод за измерване. 

<br><details>
    <summary>Ако се чудиш как да измериш разстоянието от google maps, щракни тук за да научиш!</summary>
<br>

> Долу в десния ъгъл на google maps е показан [мащаба](https://bg.wikipedia.org/wiki/%D0%9C%D0%B0%D1%89%D0%B0%D0%B1) на картата, допълнително маркиран от мен с червена стрелка. Това е отсечка върху картата, чиито реална дължина е дадена отстрани, в случая $50 \, m$. За да пресметнем реалната дължина на търсената от нас отсечка в синьо, трябва да съпоставим двете отсечки една към друга. Като се замисли човек, основното качество на една правилно начертана карта е, че всяка пройзволна отсечка $i$ начертана върху нея притежава едно и също отношение на нейната реална физическа дължина към нейната начертана дължина:
\begin{align}
    отношение = \frac{L1_{реална}}{L1_{начертана}} = \frac{L2_{реална}}{L2_{начертана}} = ... = \frac{Li_{реална}}{Li_{начертана}}. \tag{10}
\end{align}
Това отнешение можем да пресметнем благодарение дадения мащаб, като първо измерим неговата начертана дължина, а след това разделим дадената реална на начертаната дължина. В случая обаче можем да пресметнем направо реалната дължина на произволна отсечка $L1$ като я изведем от предишното уравнение $(10)$:
\begin{align}
    L1_{реална} = L1_{начертана} \cdot \frac{L2_{реална}}{L2_{начертана}} \tag{11}
\end{align}
и въведем дължините на мащаба на мястото на $L2$. Начертаната дължина в случая е дължината, която виждаме на екрана на компютъра. Просто вземи линийка и измери дължините на двете отсечки върху монитора. На моя монитор се получва следното уравнение:
\begin{align}
    L1_{реална} = 382 \, mm \cdot \frac{50 \, m}{76 \, mm} = 251.3 \, m. \tag{12}
\end{align}
На твоя монитор обаче ще измериш различно начертаните дължини в зависимост от големината и резолюцията на монитора. Освен това колкото повече увеличиш изображението, толкова по-точно ще можеш да измериш и да изчислиш реалната дължина.

</details>
<br>

<img src="bonus1/fig3.png" alt="Фигура 3" width="600"/><br>
*Фигура 3. Разстоянието от мястото на заснемане до сградата изобразено върху Google Maps.*  


---
[<<<< Връщане към Задача 1](lession1.ipynb) $~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ [Начална страница](./../welcome.ipynb) $~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ [Продължи към Бонус 2 >>>>](bonus2.ipynb)